# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 9 2022 2:22PM,250429,10,Enova-10946,Emerginnova,Released
1,Nov 9 2022 2:22PM,250429,10,Enova-10951,Emerginnova,Released
2,Nov 9 2022 2:22PM,250429,10,Eme-105718,Emerginnova,Released
3,Nov 9 2022 2:13PM,250427,10,0086153931,ISDIN Corporation,Released
4,Nov 9 2022 2:13PM,250427,10,0086153953,ISDIN Corporation,Released
5,Nov 9 2022 2:13PM,250427,10,0086153952,ISDIN Corporation,Released
6,Nov 9 2022 2:13PM,250427,10,0086153954,ISDIN Corporation,Released
7,Nov 9 2022 2:13PM,250427,10,0086153957,ISDIN Corporation,Released
8,Nov 9 2022 2:13PM,250427,10,0086153970,ISDIN Corporation,Released
9,Nov 9 2022 2:13PM,250427,10,0086153979,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,250424,Released,1
29,250425,Released,1
30,250426,Released,1
31,250427,Released,8
32,250429,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250424,NaN,NaN,1.0
250425,NaN,NaN,1.0
250426,NaN,NaN,1.0
250427,NaN,NaN,8.0
250429,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250356,0.0,0.0,1.0
250357,0.0,0.0,1.0
250364,4.0,4.0,6.0
250373,0.0,13.0,10.0
250377,19.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250356,0,0,1
250357,0,0,1
250364,4,4,6
250373,0,13,10
250377,19,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250356,0,0,1
1,250357,0,0,1
2,250364,4,4,6
3,250373,0,13,10
4,250377,19,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250356,,,1
1,250357,,,1
2,250364,4,4,6
3,250373,,13,10
4,250377,19,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 9 2022 2:22PM,250429,10,Emerginnova
3,Nov 9 2022 2:13PM,250427,10,ISDIN Corporation
11,Nov 9 2022 2:06PM,250426,10,Emerginnova
12,Nov 9 2022 2:03PM,250425,10,"Snap Medical Industries, LLC"
13,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC"
14,Nov 9 2022 1:42PM,250422,21,"NBTY Global, Inc."
15,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc."
98,Nov 9 2022 12:11PM,250411,10,Bio-PRF
111,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC"
112,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 9 2022 2:22PM,250429,10,Emerginnova,,,3
1,Nov 9 2022 2:13PM,250427,10,ISDIN Corporation,,,8
2,Nov 9 2022 2:06PM,250426,10,Emerginnova,,,1
3,Nov 9 2022 2:03PM,250425,10,"Snap Medical Industries, LLC",,,1
4,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC",,,1
5,Nov 9 2022 1:42PM,250422,21,"NBTY Global, Inc.",,,1
6,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",,,83
7,Nov 9 2022 12:11PM,250411,10,Bio-PRF,,11,2
8,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC",,1,
9,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 2:22PM,250429,10,Emerginnova,3,,
1,Nov 9 2022 2:13PM,250427,10,ISDIN Corporation,8,,
2,Nov 9 2022 2:06PM,250426,10,Emerginnova,1,,
3,Nov 9 2022 2:03PM,250425,10,"Snap Medical Industries, LLC",1,,
4,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC",1,,
5,Nov 9 2022 1:42PM,250422,21,"NBTY Global, Inc.",1,,
6,Nov 9 2022 12:43PM,250416,15,"Person & Covey, Inc.",83,,
7,Nov 9 2022 12:11PM,250411,10,Bio-PRF,2,11,
8,Nov 9 2022 11:59AM,250410,16,"Emersa Waterbox, LLC",,1,
9,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 2:22PM,250429,10,Emerginnova,3,,
1,Nov 9 2022 2:13PM,250427,10,ISDIN Corporation,8,,
2,Nov 9 2022 2:06PM,250426,10,Emerginnova,1,,
3,Nov 9 2022 2:03PM,250425,10,"Snap Medical Industries, LLC",1,,
4,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 2:22PM,250429,10,Emerginnova,3.0,NaN,NaN
1,Nov 9 2022 2:13PM,250427,10,ISDIN Corporation,8.0,NaN,NaN
2,Nov 9 2022 2:06PM,250426,10,Emerginnova,1.0,NaN,NaN
3,Nov 9 2022 2:03PM,250425,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 2:22PM,250429,10,Emerginnova,3.0,0.0,0.0
1,Nov 9 2022 2:13PM,250427,10,ISDIN Corporation,8.0,0.0,0.0
2,Nov 9 2022 2:06PM,250426,10,Emerginnova,1.0,0.0,0.0
3,Nov 9 2022 2:03PM,250425,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,Nov 9 2022 1:57PM,250424,20,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2754467,32.0,35.0,24.0
15,500780,89.0,4.0,4.0
16,751123,2.0,1.0,0.0
20,250424,1.0,0.0,0.0
21,2003133,8.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2754467,32.0,35.0,24.0
1,15,500780,89.0,4.0,4.0
2,16,751123,2.0,1.0,0.0
3,20,250424,1.0,0.0,0.0
4,21,2003133,8.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,32.0,35.0,24.0
1,15,89.0,4.0,4.0
2,16,2.0,1.0,0.0
3,20,1.0,0.0,0.0
4,21,8.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,15,Released,89.0
2,16,Released,2.0
3,20,Released,1.0
4,21,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20,21
Status,,,,,
Completed,24.0,4.0,0.0,0.0,0.0
Executing,35.0,4.0,1.0,0.0,0.0
Released,32.0,89.0,2.0,1.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20,21
0,Completed,24.0,4.0,0.0,0.0,0.0
1,Executing,35.0,4.0,1.0,0.0,0.0
2,Released,32.0,89.0,2.0,1.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20,21
0,Completed,24.0,4.0,0.0,0.0,0.0
1,Executing,35.0,4.0,1.0,0.0,0.0
2,Released,32.0,89.0,2.0,1.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()